# Teoría del Error
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para el curso de <code>Métodos Numéricos</code></p>
<style type="text/css">
    .formula {
        background: #f7f7f7;
        border-radius: 50px;
        padding: 15px;
    }
    .border {
        display: inline-block;
        border: solid 1px rgba(204, 204, 204, 0.4);
        border-bottom-color: rgba(187, 187, 187, 0.4);
        border-radius: 3px;
        box-shadow: inset 0 -1px 0 rgba(187, 187, 187, 0.4);
        background-color: inherit !important;
        vertical-align: middle;
        color: inherit !important;
        font-size: 11px;
        padding: 3px 5px;
        margin: 0 2px;
    }
</style>


## Error
El error es inherente a los métodos numéricos, por tanto es fundamental hacer seguimiento de la propagación de los errores cometidos a fin de poder estimar el grado de aproximación de la solución que se obtiene.

## Agenda
1. Error real
1. Error absoluto
1. Error relativo
1. Incertidumbre

In [1]:
# Importando librerias del cuaderno de jupyter
import math as m 
import numpy as np
import pylab as plt

# Definicion de nuevas funciones
def _significativas(valor, cifras):
    ''' Reducir un valor a un numero de cifras significativas '''
    posicion_primera_significativa = -int(m.floor(m.log10(abs(valor))))
    digitos = posicion_primera_significativa + cifras - 1
    return round(valor, digitos)

def _num_significativas(valor):
    posicion_primera_significativa = -int(m.floor(m.log10(abs(valor))))
    valor_significativo = 10**(posicion_primera_significativa)
    return 3

def _normalizar(valor, referencia):
    ''' Aproximar un numero a las cifras significativas de la referencia'''
    posicion_primera_significativa = -int(m.floor(m.log10(abs(referencia))))
    cifras_significativas = _num_significativas(referencia)
    cifras = posicion_primera_significativa + cifras_significativas
    return _significativas(valor, cifras)

## 1. Error real
---
El error real por definición es la diferencia entre el valor que se considera real y el valor aproximado, puede ser positivo si el error es por defecto, o negativo si el error es por exceso.

\begin{equation*}
E = V_r - V_a \\
\end{equation*}


In [2]:
# Modelo de gravitación Básico
# Medidas directas de longitud y ángulo en el experimento
Longitud = 1; θ = 45; deltaT = 0.30

# Medidas directas del periodo cada 30 segundos
Tr = [2.106, 2.101, 2.098, 2.087, 2.073, 2.070, 2.064, 2.059, 2.057, 2.052]

# Valores reales
Lat = 1.209673                                                      # Latitud de (Pasto - Nariño)
Alt = 2_539                                                         # Altitud de (Pasto - Nariño)
R = 6_371_000                                                       # Radio medio de la tierra
Afc = 1 + 0.0053024 * m.sin(Lat)**2 - 0.0000058 * m.sin(2*Lat)**2   # Aporte de la fuerza centrifuga
Afg = 1 - (2/R)*Alt + (3/R**2)*Alt**2                               # Aporte de la distancia al centro
g = 9.780327 * Afc * Afg

# Péndulo Simple
Ti = np.array(Tr)               # Periodo (Medida directa)
θ_rad = θ * (m.pi/180)          # Conversión del ángulo a radianes (~Medida directa)
To = Ti / (1 + (θ_rad/4)**2)    # Corrección para ángulos grandes (~Medida directa)
K = 4 * Longitud * m.pi**2      # Constante de proporcionalidad (~Medida directa)    

# Medida indirecta o parente de la graveddad
ga = K / To**2

# Error Real
Et_ga = g - ga

# Media aritmética (Promedio - P)
P_Ti = sum(Ti) / len(Ti)
P_To = sum(To) / len(To)
P_ga = sum(ga) / len(ga)
P_Et = sum(Et_ga) / len(Et_ga)

# Desviacion tipica (Desviación - D)
D_Ti = ((1/len(Ti)) * sum((P_Ti - Ti)**2))**(1/2)
D_To = ((1/len(To)) * sum((P_To - To)**2))**(1/2)
D_ga = ((1/len(ga)) * sum((P_ga - ga)**2))**(1/2)
D_Et = ((1/len(Et_ga)) * sum((P_Et - Et_ga)**2))**(1/2)

# Salida estándar
print(f"--------------------------------------------------------")
print(f"Valor del modelo algebraico: {g:8.4f}")
print(f"--------------------------------------------------------")
print(f"| No |  T [s]   |  To [s]    | g [m/s²]   | ERROR REAL |")
print(f"--------------------------------------------------------")
print(f"| No | V Medido | V aparente | V aparente | Individual |")
print(f"--------------------------------------------------------")
for N in range(len(Tr)):
    print(f"| {N+1:2} | {Ti[N]:8.3f} | {To[N]:10.3f} | {ga[N]:10.4f} | {Et_ga[N]:10.4f} |")

print(f"--------------------------------------------------------")
print(f"| P  | {P_Ti:8.3f} | {P_To:10.3f} | {P_ga:10.4f} | {P_Et:10.4f} |")
print(f"--------------------------------------------------------")
print(f"| D  | {D_Ti:8.3f} | {D_To:10.3f} | {D_ga:10.4f} | {D_Et:10.4f} |")
print(f"--------------------------------------------------------")


--------------------------------------------------------
Valor del modelo algebraico:   9.8179
--------------------------------------------------------
| No |  T [s]   |  To [s]    | g [m/s²]   | ERROR REAL |
--------------------------------------------------------
| No | V Medido | V aparente | V aparente | Individual |
--------------------------------------------------------
|  1 |    2.106 |      2.028 |     9.6006 |     0.2172 |
|  2 |    2.101 |      2.023 |     9.6464 |     0.1715 |
|  3 |    2.098 |      2.020 |     9.6740 |     0.1439 |
|  4 |    2.087 |      2.010 |     9.7762 |     0.0416 |
|  5 |    2.073 |      1.996 |     9.9087 |    -0.0909 |
|  6 |    2.070 |      1.993 |     9.9375 |    -0.1196 |
|  7 |    2.064 |      1.987 |     9.9953 |    -0.1775 |
|  8 |    2.059 |      1.983 |    10.0439 |    -0.2261 |
|  9 |    2.057 |      1.981 |    10.0635 |    -0.2456 |
| 10 |    2.052 |      1.976 |    10.1126 |    -0.2947 |
--------------------------------------------------

## 1. Error absoluto
---
El error absoluto por definición la diferencia entre el valor que se considera real y el valor aproximado, siempre es positivo; cuando se realizan n medidas de un mismo fenómeno se puede calcular como el promedio de los errores absolutos individuales o la raíz del error cuadrático medio.


\begin{align}
e_{a} & = \left| E \right| = \left| V_r-V_a \right| \\
EAM  & = \frac{1}{n}  \sum_{1}^n e_{a} =  \frac{1}{n} \sum_{1}^n \left| V_r-V_a \right| \\
RECM & =  \sqrt{ \frac{1}{n} \sum_{1}^n \left( V_r-V_a \right)^2 } \\
\end{align}


In [3]:
# Error Absoluto
ea_ga = abs(Et_ga)

# Media aritmética (Promedio - P)
P_ea = sum(ea_ga) / len(ea_ga)

# Desviacion tipica (Desviación - D)
D_ea = ((1/len(ea_ga)) * sum((P_ea - ea_ga)**2))**(1/2)

# Otros Promedios
EAM = sum(ea_ga) / len(ea_ga)
RECM = ((1/len(Et_ga)) * sum((Et_ga)**2))**(1/2)

# Salida estándar
print(f"-------------------------------------------------------------------------")
print(f"| No |  T [s]   |  To [s]    |   g[m/s²]  | ERROR REAL | ERROR ABSOLUTO |")
print(f"-------------------------------------------------------------------------")
print(f"| No | V Medido | V aparente | V aparente | Individual |   Individual   |")
print(f"-------------------------------------------------------------------------")
for N in range(len(Tr)):
    print(f"| {N+1:2} | {Ti[N]:8.3f} | {To[N]:10.3f} | {ga[N]:10.4f} | {Et_ga[N]:10.4f} | {ea_ga[N]:14.4f} |")

print(f"-------------------------------------------------------------------------")
print(f"| P  | {P_Ti:8.3f} | {P_To:10.3f} | {P_ga:10.4f} | {P_Et:10.4f} | {P_ea:14.4f} |")
print(f"-------------------------------------------------------------------------")
print(f"| D  | {D_Ti:8.3f} | {D_To:10.3f} | {D_ga:10.4f} | {D_Et:10.4f} | {D_ea:14.4f} |")
print(f"-------------------------------------------------------------------------")
print(f"Error Absoluto Medio - EAM:             {EAM:8.4f}")
print(f"Raíz Error Cuadrático Medio - RECM:     {RECM:8.4f}")
print(f"-------------------------------------------------------------------------")

-------------------------------------------------------------------------
| No |  T [s]   |  To [s]    |   g[m/s²]  | ERROR REAL | ERROR ABSOLUTO |
-------------------------------------------------------------------------
| No | V Medido | V aparente | V aparente | Individual |   Individual   |
-------------------------------------------------------------------------
|  1 |    2.106 |      2.028 |     9.6006 |     0.2172 |         0.2172 |
|  2 |    2.101 |      2.023 |     9.6464 |     0.1715 |         0.1715 |
|  3 |    2.098 |      2.020 |     9.6740 |     0.1439 |         0.1439 |
|  4 |    2.087 |      2.010 |     9.7762 |     0.0416 |         0.0416 |
|  5 |    2.073 |      1.996 |     9.9087 |    -0.0909 |         0.0909 |
|  6 |    2.070 |      1.993 |     9.9375 |    -0.1196 |         0.1196 |
|  7 |    2.064 |      1.987 |     9.9953 |    -0.1775 |         0.1775 |
|  8 |    2.059 |      1.983 |    10.0439 |    -0.2261 |         0.2261 |
|  9 |    2.057 |      1.981 |    10.0

## 3. Error relativo
---
Es el proceso de operar cifras aproximadas o truncadas

\begin{align}
e_{r} & = \frac{E}{V_r} = \frac{ \left| V_r-V_a \right| }{V_r} \\
\overline{e_{r}} & = \frac{EAM}{V_r} =  \frac{1}{n} \sum_{1}^n \frac{ \left| V_r-V_a \right| }{V_r} \\
\end{align}


In [4]:
# Error Absoluto
er_ga = 100 * ea_ga / g

# Media aritmética (Promedio - P)
P_er = sum(er_ga) / len(er_ga)

# Desviacion tipica (Desviación - D)
D_er = ((1/len(er_ga)) * sum((P_er - er_ga)**2))**(1/2)

# Otros Promedios
EPM = sum(er_ga) / len(er_ga)

# Salida estándar
print(f"------------------------------------------------------------------------------------------")
print(f"| No |  T [s]   |  To [s]    |   g[m/s²]  | ERROR REAL | ERROR ABSOLUTO | ERROR RELATIVO |")
print(f"------------------------------------------------------------------------------------------")
print(f"| No | V Medido | V aparente | V aparente | Individual |   Individual   |   Individual   |")
print(f"------------------------------------------------------------------------------------------")
for N in range(len(Tr)):
    print(f"| {N+1:2} | {Ti[N]:8.3f} | {To[N]:10.3f} | {ga[N]:10.4f} | {Et_ga[N]:10.4f} | {ea_ga[N]:14.4f} | {er_ga[N]:12.2f} % |")

print(f"------------------------------------------------------------------------------------------")
print(f"| P  | {P_Ti:8.3f} | {P_To:10.3f} | {P_ga:10.4f} | {P_Et:10.4f} | {P_ea:14.4f} | {P_er:12.2f} % |")
print(f"------------------------------------------------------------------------------------------")
print(f"| D  | {D_Ti:8.3f} | {D_To:10.3f} | {D_ga:10.4f} | {D_Et:10.4f} | {D_ea:14.4f} | {D_er:12.2f} % |")
print(f"------------------------------------------------------------------------------------------")
print(f"Error Absoluto Medio - EAM:             {EAM:8.4f}")
print(f"Raíz Porcentual Medio - EPM:            {EPM:6.2f} %")
print(f"------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------
| No |  T [s]   |  To [s]    |   g[m/s²]  | ERROR REAL | ERROR ABSOLUTO | ERROR RELATIVO |
------------------------------------------------------------------------------------------
| No | V Medido | V aparente | V aparente | Individual |   Individual   |   Individual   |
------------------------------------------------------------------------------------------
|  1 |    2.106 |      2.028 |     9.6006 |     0.2172 |         0.2172 |         2.21 % |
|  2 |    2.101 |      2.023 |     9.6464 |     0.1715 |         0.1715 |         1.75 % |
|  3 |    2.098 |      2.020 |     9.6740 |     0.1439 |         0.1439 |         1.47 % |
|  4 |    2.087 |      2.010 |     9.7762 |     0.0416 |         0.0416 |         0.42 % |
|  5 |    2.073 |      1.996 |     9.9087 |    -0.0909 |         0.0909 |         0.93 % |
|  6 |    2.070 |      1.993 |     9.9375 |    -0.1196 |         0.1196 |         1.22 % |

# Incertidumbre
---
**Incertidumbre:** Es una medida del ajuste o cálculo que debe hacerse de una cantidad con respecto al valor que se supone real de dicha magnitud; esto como una medida que refleje las características ignoradas de la magnitud o los errores cometidos para calcularla. Finalmente, esto significa que un valor aparente se puede representar expresando una cantidad como la suma de un valor confiable sumado a un intervalo de incertidumbre.

\begin{equation*}
V_a = \overline{x} \pm \Delta x \\
\end{equation*}

El valor confiable en algunos casos sepuede expresar como el _**valor promedio**_ de la cantidad a representar y la incertidumbre como la _**Desviación tipica o estándar**_, que es una medida del ajuste o cálculo de una magnitud con respecto al valor que se supone real de dicha magnitud.

\begin{align}
V_a & = \mu \pm \sigma \\
\mu & =  \frac{1}{n} \sum_{1}^n x_i \\
\sigma & = \sqrt{ \frac{1}{n} \sum_{1}^n \left( \mu - x_i \right)^2 } \\
\end{align}

In [5]:
# Valor aparente del periodo
μT = sum(To) / len(To)              # Valor promedio
DT = sum((μT - To)**2) / len(To)    # Valor promedio

# Valor aparente de la gravedad
μg = P_ga                           # Valor promedio
Dg = sum((μg - ga)**2) / len(ga)    # Incertidumbre

# Una convencion aceptada es expresar la incertidumbre en una sola cifra significativa
ΔT = _significativas(DT, 1)
Δg = _significativas(Dg, 1)

# Valores medios normalizados
μT_n = _normalizar(μT, ΔT)
μg_n = _normalizar(μg, Δg)

# Salida estandar
print(f"-----------------------------------------------")
print(f"Valor promedio de T:           {μT:8.4f}")
print(f"Incertidumbre del grupo T:     {DT:8.4f}")
print(f"Valor aparente del grupo:      {μT_n:8.4f} ± {ΔT}")
print(f"-----------------------------------------------")
print(f"Valor promedio de g:           {μg:8.4f}")
print(f"Incertidumbre del grupo g:     {Dg:8.4f}")
print(f"Valor aparente del grupo:      {μg_n:8.4f} ± {Δg}")
print(f"-----------------------------------------------")

-----------------------------------------------
Valor promedio de T:             1.9996
Incertidumbre del grupo T:       0.0003
Valor aparente del grupo:        1.9996 ± 0.0003
-----------------------------------------------
Valor promedio de g:             9.8759
Incertidumbre del grupo g:       0.0318
Valor aparente del grupo:        9.8759 ± 0.03
-----------------------------------------------


---
## Mas Recursos

- [Métodos de redondeo](https://en.wikipedia.org/wiki/Rounding) (Wikipedia)
- [Error experimental](https://es.wikipedia.org/wiki/Error_experimental) (Wikipedia)
- [Error de aproximación](https://es.wikipedia.org/wiki/Error_de_aproximaci%C3%B3n) (Wikipedia)
- [Error absoluto medio](https://es.wikipedia.org/wiki/Error_absoluto_medio) (Wikipedia)

